# ULTImodel tutorial 5: Finalize attributes for trip generation and distribution

In this tutorial, the attraction factors per TAZ that influence trip generation and distribution will be determined using the `AttractionFactors` methods.

The inputs are:
- TAZ as GeoDataFrame
- Population density as point GeoDataFrame

In [1]:
# for network creation
from ultimodel import AttractionFactors

In [2]:
# for reading and saving files etc.
import geopandas as gpd
import pandas as pd
import numpy as np
# for time tracking
from datetime import datetime

## Read input: TAZ and population

The input includes georeferenced TAZ in `EPSG:4326` with the following _required_ attributes (columns):

* __ID__ | field including a unique ID, e.g. the NUTS ID
* __Country__ | field containing the ISO-2 code of the respective country

Other attributes like name etc. can be added, but are not required for the following steps.

To determine population per TAZ, a GeoDataFrame containing points with population on a 1km grid is used, based on [GHSL](https://ghsl.jrc.ec.europa.eu/download.php?ds=pop). 

In [3]:
# load taz from database or local
taz = gpd.GeoDataFrame.from_file('tutorial-files/_input/taz-tutorial.gpkg')

# defining ID, country and geometry column names of taz
taz_id = "nuts_id"
taz_cn = "cntr_code"
taz_geo = "geometry"

taz.head()

,nuts_id,cntr_code,nuts_name,geometry
0,FI193,FI,Keski-Suomi,"MULTIPOLYGON (((26.13865 63.45759, 26.16055 63..."
1,FI194,FI,Etelä-Pohjanmaa,"MULTIPOLYGON (((21.64783 62.01959, 21.63880 62..."
2,FI195,FI,Pohjanmaa,"MULTIPOLYGON (((21.64783 62.01959, 21.52578 62..."
3,FI196,FI,Satakunta,"MULTIPOLYGON (((21.41993 61.04744, 21.42015 61..."
4,FI197,FI,Pirkanmaa,"MULTIPOLYGON (((22.83124 62.27089, 22.90118 62..."


In [4]:
# read population
pop = gpd.GeoDataFrame.from_file('tutorial-files/_input/4326-points-population-dk.gpkg')

## Attraction factors on cell level

For the attraction factors on cell level, which will be used for trip generation and distribution, __population__ and __industry__ data is aggregated per TAZ. This information will then be transformed to an attraction index for

- national distribution
- international distribution

In [5]:
path_export = 'tutorial-files/tutorial5/'

In [6]:
att = AttractionFactors.AttractionIndex(taz[taz[taz_cn]=='DK'])

### Population

Get population per TAZ based on population density from a `Point GeoDataFrame`

In [7]:
# population
print("... get population {}".format(datetime.now()))
att.population_from_point(pop)
print("finished at {}".format(datetime.now()))

... get population 2023-04-13 16:56:54.669761
finished at 2023-04-13 16:56:56.268585


### Industry

Get industry areas from OSM and aggreate total industrial area, number of industrial areas per TAZ

In [8]:
# industry
print("... get industrial sites from OSM {}".format(datetime.now()))
att.industry_attributes_from_osm()
print("finished at {}".format(datetime.now()))

... get industrial sites from OSM 2023-04-13 16:56:56.278315


11it [00:13,  1.23s/it]

finished at 2023-04-13 16:57:09.799890


### Calculate attraction index

The attraction index is calculated per TAZ using the values for population, industrial site count and aggregated area in relation to the mean. This means, the index for national transport is different to the international index. Hence, we set the scope in the `att.attraction_index()` function for the national index and leave is as `None` for the international index.

In [9]:
taz_cn = att.attraction_index(scope='DK')
taz_cn.head()

,nuts_id,cntr_code,nuts_name,geometry,population,ind_area_count,ind_area_sum,index_nat
0,DK011,DK,Byen København,"MULTIPOLYGON (((12.50479 55.63734, 12.48025 55...",1.199214e+06,58,6.274909e+06,0.580681
1,DK012,DK,Københavns omegn,"MULTIPOLYGON (((12.24627 55.70664, 12.25874 55...",8.048375e+05,86,2.605555e+07,0.749349
2,DK013,DK,Nordsjælland,"MULTIPOLYGON (((12.24627 55.70664, 12.21994 55...",6.529701e+05,135,1.572232e+07,0.665871
3,DK014,DK,Bornholm,"MULTIPOLYGON (((14.88943 55.22813, 14.94373 55...",4.738833e+04,52,2.787603e+06,0.091701
4,DK021,DK,Østsjælland,"MULTIPOLYGON (((12.24627 55.70664, 12.16537 55...",3.613609e+05,75,1.117585e+07,0.392678


In [10]:
taz_cn.to_file(path_export + 'taz-tutorial-attindex-dk' + '.gpkg', driver='GPKG')

## Multiple Countries

In addition to an index for national transport, an international index can be created where the attraction factors of all taz in the regarded region are created.

International

In [11]:
# get international index
taz_int = gpd.GeoDataFrame.from_file(path_export + '_import/taz-tutorial-int.gpkg')
att_int = AttractionFactors.AttractionIndex(taz_int)
taz_int = att.attraction_index()
taz_int.head()

,nuts_id,cntr_code,nuts_name,geometry,population,ind_area_count,ind_area_sum,index_int
0,DK011,DK,Byen København,"MULTIPOLYGON (((12.50479 55.63734, 12.48025 55...",1.199214e+06,58,6.274909e+06,0.580681
1,DK012,DK,Københavns omegn,"MULTIPOLYGON (((12.24627 55.70664, 12.25874 55...",8.048375e+05,86,2.605555e+07,0.749349
2,DK013,DK,Nordsjælland,"MULTIPOLYGON (((12.24627 55.70664, 12.21994 55...",6.529701e+05,135,1.572232e+07,0.665871
3,DK014,DK,Bornholm,"MULTIPOLYGON (((14.88943 55.22813, 14.94373 55...",4.738833e+04,52,2.787603e+06,0.091701
4,DK021,DK,Østsjælland,"MULTIPOLYGON (((12.24627 55.70664, 12.16537 55...",3.613609e+05,75,1.117585e+07,0.392678


In [12]:
taz_int.to_file(path_export + 'taz-tutorial-attindex-int' + '.gpkg', driver='GPKG')